In [1]:
# THE PURPOSE OF THIS SCRIPT IS TO: 

# 1) TAKE IN MIL MILAGROS'S FULL DATASET (CONTAINING A ROW FOR EVERY CHECK-UP) AND THE INDIVIDUAL DATASET (CONTAINING A ROW FOR EVERY
#    CHILD), AND DERIVE ANOTHER TABLE THAT DISPLAYS COMMUNITY-WIDE INFORMATION, SUCH AS YEARS AFFILIATED WITH MIL MILAGROS, TOTAL NUMBER 
#    OF KIDS AND CHECK-UPS, AVERAGE CHANGE IN HEIGHT-FOR-AGE Z-SCORE AMONG ITS KIDS, ETC.

# 2) RUN A STATISTICAL TEST THAT INVESTIGATES WHETHER ANY OF THE COMMUNITIES DIFFER FROM EACH OTHER IN THEIR AVERAGE CHANGE IN HEIGHT-FOR-AGE Z-SCORE  

In [2]:
# IMPORTING PACKAGES
import pandas as pd
import statsmodels.api as sm
from statsmodels.sandbox.stats.multicomp import MultiComparison
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# READING IN DATASET WITH ONE ROW FOR EVERY CHECK-UP
mm_data = pd.read_excel("/Users/ajarbuckle/Desktop/MM PROJECT/mm_data_redo/mm_data_clean.xlsx") 
mm_data

,nombre_de_la_madre,nombre_del_niño/niña,fecha_de_nacimiento,sexo,comunidad,fecha_de_monitoreo,peso,talla,edad_a,edad_m,edad_s,ID,puntaje_z
0,Ofelia Gabina Alva Mus,Samuel David Alva Alva,2015-09-19,M,Pahaj,2017-12-06,8.7,76.4,2.214921,26,115,2,-3.906698
1,María Teresa Cos González,Jordy Levni Alfredo López Cos,2016-08-10,M,Pahaj,2019-01-16,13.3,81.0,2.433949,29,127,7,-3.039551
2,Elsa Griselda Xaminez,Kerman Eliel González Xaminez,2016-03-17,M,Pamezabal,2019-01-16,12.4,85.5,2.833676,34,147,8,-2.561794
3,Cristina Godoy,Eduardo Emilio Ixcol Godoy,2015-09-21,M,Los Manantiales,2019-04-10,12.2,86.0,3.550992,42,185,11,-3.493355
4,Ana Cristina Chox Ixcol,Domingo Braxton Tay Chox,2014-06-12,M,Los Manantiales,2019-01-16,15.8,97.3,4.596851,55,239,20,-2.228093
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3779,Carolina Estefany Sajché Zavala,Heredyt Tatiana Noj Sajche,2019-09-10,F,Pahaj,2019-12-11,5.5,58.4,0.251882,3,13,526,-0.654548
3780,Francisca Hilaria Bac,Manuela Claret Tulul,2018-10-24,F,Pahaj,2019-01-18,5.4,55.1,0.235455,2,12,495,-1.958459
3781,Santos Lucrecia López Ixcol de Aju,Nena Recién nacida,2019-02-26,F,Ciénaga Grande,2019-03-13,2.5,43.0,0.041068,0,2,530,-4.473173
3782,Santos Lucrecia López Ixcol de Aju,Nena Recién nacida,2019-02-26,F,Ciénaga Grande,2019-04-10,2.6,45.0,0.117728,1,6,530,-5.064258


In [3]:
# CREATING A TEMPORARY TABLE THAT HAS COMMUNITY-WIDE INFORMATION ON INFORMATION NOT RELATED TO INDIVIDUAL CHILDREN
temp1 = mm_data.groupby('comunidad').agg(
    comunidad = ('comunidad', 'first'),
    año_primero = ('fecha_de_monitoreo', lambda x: x.min().year),
    año_último = ('fecha_de_monitoreo', lambda x: x.max().year),
    chequeos_total = ('ID', 'count'), 
    niños_total = ('ID', lambda x: x.nunique())
    )
temp1

,comunidad,año_primero,año_último,chequeos_total,niños_total
comunidad,,,,,
Campo Verde,Campo Verde,2018,2019,25,9
Chuijomil,Chuijomil,2018,2021,218,42
Ciénaga Grande,Ciénaga Grande,2019,2021,117,39
Cruz B,Cruz B,2019,2021,301,61
Desconocido,Desconocido,2017,2018,12,6
Los Manantiales,Los Manantiales,2018,2021,213,31
Los Planes,Los Planes,2017,2021,243,43
Nikajkim,Nikajkim,2019,2021,281,52
Nuevo Progreso,Nuevo Progreso,2014,2021,226,36


In [4]:
# READING IN DATASET WITH ONE ROW FOR EVERY CHILD
mm_data = pd.read_excel("/Users/ajarbuckle/Desktop/MM PROJECT/mm_data_redo/mm_data_individual_table.xlsx") 
mm_data

,ID,nombre_del_niño,fecha_de_nacimiento,comunidad,sexo,monitoreo_prim,monitoreo_ult,talla_prim,talla_ult,z_prim,z_ult,observaciones,talla_dif,z_dif,monitoreo_prim_edad,duración,z_dif_rate,frecuencia,stunted_at_first,stunted_at_end
0,1,Maybelin Alicia Aju López,2016-02-22,Los Planes,F,2017-08-15,2019-10-25,70.5,87.3,-3.219024,-3.135741,10,16.8,0.083283,1.478439,2.193018,0.037976,4.559925,True,True
1,2,Samuel David Alva Alva,2015-09-19,Pahaj,M,2017-12-06,2020-01-15,76.4,92.5,-3.906698,-2.904418,11,16.1,1.002280,2.214921,2.108145,0.475432,5.217857,True,True
2,3,Miriam Tulul Bac,2017-01-25,Pahaj,F,2018-10-19,2021-12-08,72.5,95.6,-3.386637,-2.724794,9,23.1,0.661844,1.730322,3.137577,0.210941,2.868455,True,True
3,4,Cristian Anibal Avila Chávez,2019-03-21,Xesampual,M,2019-07-03,2021-12-08,62.0,90.0,0.279202,-0.961131,16,28.0,-1.240333,0.284736,2.433949,-0.509597,6.573678,False,False
4,5,Diego Alejandro Tzaj Chox,2019-01-08,Xesampual,M,2019-05-15,2019-07-31,56.4,61.0,-3.597655,-3.094706,4,4.6,0.502948,0.347707,0.210815,2.385739,18.974026,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,535,Erikca Maria Rosalina Leal Quisquina,2016-09-28,Ciénaga Grande,F,2019-03-13,2019-04-10,84.7,85.8,-1.493608,-1.382273,2,1.1,0.111335,2.453114,0.076660,1.452326,26.089286,False,False
420,536,Rudy Gabriel Ajú López,2015-12-22,Ciénaga Grande,M,2019-03-13,2019-04-10,85.6,85.8,-3.100616,-3.178239,2,0.2,-0.077623,3.222450,0.076660,-1.012568,26.089286,True,True
421,538,Nataly Ileana Leal Quisquina,2014-08-06,Ciénaga Grande,F,2019-03-13,2019-04-10,99.2,99.0,-1.646368,-1.795524,2,-0.2,-0.149155,4.599589,0.076660,-1.945679,26.089286,False,False
422,544,Eduardo Valeriano Chávez,2015-07-13,Nikajkim,M,2019-03-13,2019-04-24,97.0,97.2,-0.861285,-1.082725,2,0.2,-0.221440,3.665982,0.114990,-1.925733,17.392857,False,False


In [5]:
# CREATING A TEMPORARY TABLE THAT HAS COMMUNITY-WIDE INFORMATION ON INFORMATION RELATED TO INDIVIDUAL CHILDREN
temp2 = mm_data.groupby('comunidad').agg(
    comunidad = ('comunidad', 'first'),
    avg_num_check = ('observaciones', 'mean'),
    avg_duration = ('duración', 'mean'),
    avg_freq = ('frecuencia', 'mean'),
    avg_start_age = ('monitoreo_prim_edad', 'mean'),
    avg_start_z = ('z_prim', 'mean'),
    avg_end_z = ('z_ult', 'mean')
    )

# CREATING A NEW COLUMN IN TEMP2 FOR AVERAGE CHANGE IN HEIGHT-FOR-AGE Z-SCORE PER CHILD
temp2['avg_change_z'] = temp2['avg_end_z'] - temp2['avg_start_z']

# CREATING A COLUMN FOR THE RATE OF HEIGHT-FOR-AGE Z-SCORE CHANGE OVER TIME
temp2['avg_z_dif_rate'] = (temp2['avg_change_z'] / temp2['avg_duration'])

temp2

,comunidad,avg_num_check,avg_duration,avg_freq,avg_start_age,avg_start_z,avg_end_z,avg_change_z,avg_z_dif_rate
comunidad,,,,,,,,,
Campo Verde,Campo Verde,2.625000,0.365503,7.446676,3.220397,-1.313437,-1.310037,0.003400,0.009302
Chuijomil,Chuijomil,6.964286,1.226655,7.327325,1.568397,-2.156903,-2.327209,-0.170305,-0.138837
Ciénaga Grande,Ciénaga Grande,4.523810,0.637007,14.100855,2.098106,-2.138112,-2.112634,0.025478,0.039996
Cruz B,Cruz B,7.179487,1.166746,8.770915,1.276400,-1.754031,-2.046797,-0.292766,-0.250925
Desconocido,Desconocido,6.500000,2.123203,3.184562,2.510609,0.089988,-1.354337,-1.444325,-0.680258
Los Manantiales,Los Manantiales,7.384615,1.278050,9.275086,1.727057,-1.838661,-2.176515,-0.337855,-0.264352
Los Planes,Los Planes,8.360000,1.682355,7.282634,1.462998,-1.973983,-2.348819,-0.374836,-0.222804
Nikajkim,Nikajkim,6.837838,1.255046,7.797931,1.651812,-1.646646,-1.681127,-0.034481,-0.027474
Nuevo Progreso,Nuevo Progreso,8.076923,1.586585,6.288308,1.295003,-1.716160,-2.364362,-0.648201,-0.408551


In [6]:
# PUTTING THE TWO TEMPORARY TABLES TOGETHER TO ARRIVE AT THE FINAL DATAFRAME
# IT WAS NECESSARY TO FIRST RESET THE INDICES OF THE TEMPORARY TABLES SO THE MERGE FUNCTION WOULD WORK
temp1 = temp1.reset_index(drop=True)
temp2 = temp2.reset_index(drop=True)
final_table = pd.merge(temp1, temp2, left_on='comunidad', right_on='comunidad', how = 'outer')

# ROUNDING COLUMN VALUES TO MAKE THE FINAL TABLE LOOK NICER 
columns_to_round = ['avg_num_check', 'avg_duration', 'avg_freq', 'avg_start_age', 'avg_start_z', 'avg_end_z', 'avg_change_z', 'avg_z_dif_rate']
final_table[columns_to_round] = final_table[columns_to_round].round(2)

final_table

,comunidad,año_primero,año_último,chequeos_total,niños_total,avg_num_check,avg_duration,avg_freq,avg_start_age,avg_start_z,avg_end_z,avg_change_z,avg_z_dif_rate
0,Campo Verde,2018,2019,25,9,2.62,0.37,7.45,3.22,-1.31,-1.31,0.00,0.01
1,Chuijomil,2018,2021,218,42,6.96,1.23,7.33,1.57,-2.16,-2.33,-0.17,-0.14
2,Ciénaga Grande,2019,2021,117,39,4.52,0.64,14.10,2.10,-2.14,-2.11,0.03,0.04
3,Cruz B,2019,2021,301,61,7.18,1.17,8.77,1.28,-1.75,-2.05,-0.29,-0.25
4,Desconocido,2017,2018,12,6,6.50,2.12,3.18,2.51,0.09,-1.35,-1.44,-0.68
5,Los Manantiales,2018,2021,213,31,7.38,1.28,9.28,1.73,-1.84,-2.18,-0.34,-0.26
6,Los Planes,2017,2021,243,43,8.36,1.68,7.28,1.46,-1.97,-2.35,-0.37,-0.22
7,Nikajkim,2019,2021,281,52,6.84,1.26,7.80,1.65,-1.65,-1.68,-0.03,-0.03
8,Nuevo Progreso,2014,2021,226,36,8.08,1.59,6.29,1.30,-1.72,-2.36,-0.65,-0.41
9,Pahaj,2014,2021,1638,209,9.68,1.74,7.07,1.27,-2.19,-2.36,-0.16,-0.09


In [7]:
# NOW GOING BACK TO THE DATASET WITH ONE ROW PER CHILD TO RUN THE STATISTICAL TEST TO SEE IF THERE ARE STATISTICALLY SIGNIFICANT DIFFERENCES IN RATE OF CHANGE 
# OF HEIGHT-FOR-AGE Z-SCORE OVER TIME BETWEEN COMMUNITIES 

mm_data = pd.read_excel("/Users/ajarbuckle/Desktop/MM PROJECT/mm_data_redo/mm_data_individual_table.xlsx") 
mm_data

,ID,nombre_del_niño,fecha_de_nacimiento,comunidad,sexo,monitoreo_prim,monitoreo_ult,talla_prim,talla_ult,z_prim,z_ult,observaciones,talla_dif,z_dif,monitoreo_prim_edad,duración,z_dif_rate,frecuencia,stunted_at_first,stunted_at_end
0,1,Maybelin Alicia Aju López,2016-02-22,Los Planes,F,2017-08-15,2019-10-25,70.5,87.3,-3.219024,-3.135741,10,16.8,0.083283,1.478439,2.193018,0.037976,4.559925,True,True
1,2,Samuel David Alva Alva,2015-09-19,Pahaj,M,2017-12-06,2020-01-15,76.4,92.5,-3.906698,-2.904418,11,16.1,1.002280,2.214921,2.108145,0.475432,5.217857,True,True
2,3,Miriam Tulul Bac,2017-01-25,Pahaj,F,2018-10-19,2021-12-08,72.5,95.6,-3.386637,-2.724794,9,23.1,0.661844,1.730322,3.137577,0.210941,2.868455,True,True
3,4,Cristian Anibal Avila Chávez,2019-03-21,Xesampual,M,2019-07-03,2021-12-08,62.0,90.0,0.279202,-0.961131,16,28.0,-1.240333,0.284736,2.433949,-0.509597,6.573678,False,False
4,5,Diego Alejandro Tzaj Chox,2019-01-08,Xesampual,M,2019-05-15,2019-07-31,56.4,61.0,-3.597655,-3.094706,4,4.6,0.502948,0.347707,0.210815,2.385739,18.974026,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,535,Erikca Maria Rosalina Leal Quisquina,2016-09-28,Ciénaga Grande,F,2019-03-13,2019-04-10,84.7,85.8,-1.493608,-1.382273,2,1.1,0.111335,2.453114,0.076660,1.452326,26.089286,False,False
420,536,Rudy Gabriel Ajú López,2015-12-22,Ciénaga Grande,M,2019-03-13,2019-04-10,85.6,85.8,-3.100616,-3.178239,2,0.2,-0.077623,3.222450,0.076660,-1.012568,26.089286,True,True
421,538,Nataly Ileana Leal Quisquina,2014-08-06,Ciénaga Grande,F,2019-03-13,2019-04-10,99.2,99.0,-1.646368,-1.795524,2,-0.2,-0.149155,4.599589,0.076660,-1.945679,26.089286,False,False
422,544,Eduardo Valeriano Chávez,2015-07-13,Nikajkim,M,2019-03-13,2019-04-24,97.0,97.2,-0.861285,-1.082725,2,0.2,-0.221440,3.665982,0.114990,-1.925733,17.392857,False,False


In [8]:
# Create a MultiComparison object, then running a Tukey's HSD test
mc = MultiComparison(mm_data['z_dif_rate'], mm_data['comunidad'])
result = mc.tukeyhsd()
print(result)

         Multiple Comparison of Means - Tukey HSD, FWER=0.05         
     group1          group2     meandiff p-adj   lower  upper  reject
---------------------------------------------------------------------
    Campo Verde       Chuijomil  -0.3855 0.9942 -1.5279  0.757  False
    Campo Verde  Ciénaga Grande   0.0701    1.0 -1.1139 1.2541  False
    Campo Verde          Cruz B  -0.3634 0.9954 -1.4695 0.7427  False
    Campo Verde     Desconocido  -0.5781 0.9995 -2.8311 1.6749  False
    Campo Verde Los Manantiales  -0.0284    1.0 -1.1806 1.1238  False
    Campo Verde      Los Planes  -0.1207    1.0 -1.2783 1.0369  False
    Campo Verde        Nikajkim   0.0161    1.0 -1.0951 1.1272  False
    Campo Verde  Nuevo Progreso  -0.4356 0.9851 -1.5878 0.7166  False
    Campo Verde           Pahaj  -0.1711    1.0 -1.2029 0.8608  False
    Campo Verde       Pamezabal   0.0343    1.0 -1.1233 1.1919  False
    Campo Verde       Xesampual    0.117    1.0 -1.0528 1.2867  False
      Chuijomil  Cié

In [9]:
# ALL VALUES FOR 'REJECT' ARE FALSE, MEANING THAT THERE ARE NO STATISTICALLY SIGNIFICANT DIFFERENCES FOR THE RATE OF CHANGE OF HEIGHT-FOR-AGE Z-SCORES
# BETWEEN THE COMMUNITIES 